In [1]:
#%pip install azure-ai-projects

## Example of using three Azure AI agents with AUTOGEN as the orchestrator

In [2]:

from PIL import Image
from pathlib import Path
from IPython.display import display, HTML
import os
import sys
import time


from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

In [3]:
from azure.identity import DefaultAzureCredential

In [4]:
from azure.ai.agents import AgentsClient 
from azure.ai.agents.models import CodeInterpreterTool, BingGroundingTool
import os
import yaml

from dotenv import load_dotenv
load_dotenv()

True

In [5]:
with open("model_config.yaml", "r") as f:
        model_config = yaml.safe_load(f)
model_client = AzureOpenAIChatCompletionClient.load_component(model_config)

In [6]:
endpoint = os.getenv("PROJECT_ENDPOINT")
print(f"Using endpoint: {endpoint}")
credential = DefaultAzureCredential()

project_client = AgentsClient(endpoint=endpoint, credential=credential)

Using endpoint: https://aq-ai-foundry-sweden-central.services.ai.azure.com/api/projects/firstProject


In [7]:
project_client = AgentsClient(
    credential=DefaultAzureCredential(),
    endpoint=endpoint
)

In [8]:
# [START create_agent_with_bing_grounding_tool]
conn_id = os.getenv("AZURE_BING_CONNECTION_ID")

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

# define the web_ai_agent, with BING grounding

In [9]:
async def web_ai_agent(query: str) -> str:
    print()
    print("This is Bing for Azure AI Agent Service .......")
    print()
    bing = BingGroundingTool(connection_id=conn_id)
    # with project_client:
    agent = project_client.create_agent(
            model="gpt-4.1-mini",
            name="my-web-agent",
            instructions="""        
                You are a web search agent.
                Your only tool is search_tool - use it to find information.
                You make only one search call at a time.
                Once you have the results, you never do calculations based on them.
            """,
            tools=bing.definitions,
            headers={"x-ms-enable-preview": "true"}
        )
    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.messages.create(
            thread_id=thread.id,
            role="user",
            content=query,
    )
    print(f"SMS: {message}")
        # Create and process agent run in thread with tools
    run = project_client.runs.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

        # Delete the assistant when done
    #project_client.agents.delete_agent(agent.id)
    #print("Deleted agent")

        # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print("Messages:"+ messages["data"][0]["content"][0]["text"]["value"])

        # project_client.close()

    return messages["data"][0]["content"][0]["text"]["value"]



# Define the blog writer agent with Code Interpreter tool

In [10]:
async def save_blog_agent(blog_content: str) -> str:
    print()
    print("This is Code Interpreter for Azure AI Agent Service .......")


    code_interpreter = CodeInterpreterTool()
        
    agent = project_client.agents.create_agent(
            model="gpt-4.1-mini",
            name="my-coder-agent",
            instructions="You are helpful agent",
            tools=code_interpreter.definitions,
            # tool_resources=code_interpreter.resources,
    )

    thread = project_client.messages.threads.create()

    message = project_client.messages.create(
            thread_id=thread.id,
            role="user",
            content="""
        
                    You are my Python programming assistant. Generate code,save """+ blog_content +
                    
                """    
                    and execute it according to the following requirements

                    1. Save blog content to blog-{YYMMDDHHMMSS}.md

                    2. give me the download this file link
                """,
    )
    # create and execute a run
    run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
            # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

        # # delete the original file from the agent to free up space (note: this does not delete your version of the file)
        # project_client.agents.delete_file(file.id)
        # print("Deleted file")

        # print the messages from the agent
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

        # get the most recent message from the assistant
        # In the save_blog_agent function, update to:
    last_message_text = None
    for msg in reversed(messages["data"]):
            if msg.get("role") == "assistant":
                last_message_text = msg["content"][0]["text"]["value"]
                break
    if last_message_text:
            print(f"Last Message: {last_message_text}")


    for file_path_annotation in messages.file_path_annotations:

        file_name = os.path.basename(file_path_annotation.text)

        project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./blog")
        print(f"File saved: {file_name}")


    #project_client.agents.delete_agent(agent.id)
    #print("Deleted agent")


        # project_client.close()


    return "Saved"


    
    

## Define the three agents

In [11]:
bing_search_agent = AssistantAgent(
    name="bing_search_agent",
    model_client=model_client,
    tools=[web_ai_agent],
    system_message="You are a search expert, help me use tools to find relevant knowledge",
)


write_agent = AssistantAgent(
    name="write_agent",
    model_client=model_client,
    system_message="""
        You are a blog writer, please help me write a blog based on bing search content."
    """
)

save_blog_content_agent = AssistantAgent(
    name="save_blog_content_agent",
    model_client=model_client,
    tools=[save_blog_agent],
    system_message="""
        Save blog content. Respond with 'Saved' to when the blog is saved.
    """
)



In [12]:
text_termination = TextMentionTermination("Saved")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(10)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination

# Define the AUTOGEN team of Azure AI agents

In [13]:

reflection_team = RoundRobinGroupChat([bing_search_agent, write_agent,save_blog_content_agent], termination_condition=termination)

## start the team chat

In [14]:
await Console(
    reflection_team.run_stream(task="""

                    I am writing one blog about machine learning. Search for the following 3 questions and write a blog based on the search results ,save it, give me the download this file link
                    
                    1. What is Machine Learning?
                    2. The difference between AI and ML
                    3. The history of Machine Learning
                    4. The future of Machine Learning
                    
                    
    """)
)  # Stream the messages to the console.

---------- TextMessage (user) ----------


                    I am writing one blog about machine learning. Search for the following 3 questions and write a blog based on the search results ,save it, give me the download this file link

                    1. What is Machine Learning?
                    2. The difference between AI and ML
                    3. The history of Machine Learning
                    4. The future of Machine Learning


    

This is Bing for Azure AI Agent Service .......

Created agent, ID: asst_dFh7JPxpC3mURAdndhsnZCCq
Created thread, ID: thread_sUZZrm99vRWs3lWL2m8I3Z5x
SMS: {'id': 'msg_0xqICHV2ontHe2Ad3Aj7wLEl', 'object': 'thread.message', 'created_at': 1750181680, 'assistant_id': None, 'thread_id': 'thread_sUZZrm99vRWs3lWL2m8I3Z5x', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'What is Machine Learning?', 'annotations': []}}], 'attachments': [], 'metadata': {}}

This is Bing for Azure AI Agent Service .......

Created

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='\n\n                    I am writing one blog about machine learning. Search for the following 3 questions and write a blog based on the search results ,save it, give me the download this file link\n\n                    1. What is Machine Learning?\n                    2. The difference between AI and ML\n                    3. The history of Machine Learning\n                    4. The future of Machine Learning\n\n\n    ', type='TextMessage'), ToolCallRequestEvent(source='bing_search_agent', models_usage=RequestUsage(prompt_tokens=140, completion_tokens=91), metadata={}, content=[FunctionCall(id='call_1VqGV8pL8jGLi9hbK3oB6YkX', arguments='{"query": "What is Machine Learning?"}', name='web_ai_agent'), FunctionCall(id='call_eq4wdWkvWXrjRYfWhUBLcgg6', arguments='{"query": "difference between AI and ML"}', name='web_ai_agent'), FunctionCall(id='call_f4V96GWaF8yczIbbFhc2FJ55', arguments='{"query": "h